Here's the final output.

The `AsBool` column returns the same value that you'd get from running 
it with the switch: `-AsBool`
```ps1
Test-IsBlank $doesNotExist -AsBool
    # true
```

### **tip** `IsNullOrEmpty` is a shortcut

These two functions are super versatile. You can pass things that aren't strings and it won't error.


```ps1
[string]::IsNullOrEmpty( $value )
[string]::IsNullOrWhiteSpace( $value )
```

### Simple Test
```ps1
$examples = $null, ' ' , '' 
```
results
```
IsNull:     True,  False, False
IsEmptyStr: False, False, True
IsBlank:    True,  True,  True
```

### Full Test

```ps1
IsTrueNull IsEmpty IsTrueEmptyStr IsBlank     Length RawValue AsBool Name
---------- ------- -------------- -------     ------ -------- ------ ----
     False    True          False    True         15 {}         True TrueArray.Empty
     False    True           True    True <EmptyStr>            True String.TrueBlank
     False   False          False    True          2            True String.Space
     False   False          False   False          2  4        False String.WithNonWhitespaceValue
      True    True          False    True     <null>            True TrueNull
     False   False          False    True          6 …          True Whitespace
      True    True          False    True     <null>            True Variable.DoesNotExist
      True    True          False    True     <null>            True Member.DoesNotExist
```

Why does `$Null` have to be on the *left hand side* when testing for `nulls?`

See: [PsScriptAnalyzer/rules/PossibleIncorrectComparisonWithNull](https://learn.microsoft.com/en-us/powershell/utility-modules/psscriptanalyzer/rules/possibleincorrectcomparisonwithnull?view=ps-modules)

In [18]:
# First, simple versions

function IsNull {
    # test for true null only. everything else is false
    [OutputType('System.Boolean')]
    param( $Object  )
    $null -eq $Object
}
function IsEmptyStr {
    # test for empty string only. everything else is false
    [OutputType('System.Boolean')]
    param( $Object  )
    ($Object -is [string]) -and $Object.length -eq ''
}
function IsBlank {
    [OutputType('System.Boolean')]
    param( $Object )
    [string]::IsNullOrWhiteSpace( $Object )
}

$examples = $null, ' ' , ''
$examples | %{
    IsNull $_
} | Join-String -sep ', ' -op 'IsNull:     '

# output
# true, false, false

$examples | %{
    IsEmptyStr $_
} | Join-String -sep ', ' -op 'IsEmptyStr: '

# output
# false, false, true

$examples | %{
    IsBlank $_
} | Join-String -sep ', ' -op 'IsBlank:    '

# output
# true, true, true

IsNull:     True, False, False
IsEmptyStr: False, False, True
IsBlank:    True, True, True


In [13]:

function Test-IsBlank {
    <#
    .synopsis
        Checks if an object is a true null. starting to get fancy
    .description
        Creates a table of different kinds of blankness
        - true null values
        - true empty string
        - strings with whitespace
    #>
    param(
        $Obj,
        # if true, the function returns a single boolean, no info
        [switch]$AsBool
    )
    $filtered = $Obj -replace "`a", '' # otherwise ascii bell is not "whitespace"
    if($AsBool) {
       return [string]::IsNullOrWhiteSpace( $filtered )
    }

    $isTrueNull     = $Null -eq $Obj
    $isStr          = $Obj -is [String]
    $isTrueEmptyStr = $isStr -and ($Obj.Length -eq 0)

    $finalLength = if(-not $IsTrueNull) { $Obj.ToString().Length }
    if($isTrueEmptyStr) { $finalLength = '<EmptyStr>' }
    if($IsTrueNull)     { $finalLength = '<null>'     }

    [pscustomobject]@{
        IsTrueNull     = $isTrueNull
        IsEmpty        = [string]::IsNullOrEmpty( $Obj )
        IsTrueEmptyStr = $isTrueEmptyStr
        IsBlank        = [string]::IsNullOrWhiteSpace( $filtered )
        Length         = $finalLength
        RawValue       = $Obj
        AsBool         = [string]::IsNullOrWhiteSpace( $filtered )
    }
}

In [12]:
# pwsh classes are useful if you want a record type
# think of a case where you have a list of [pscustomobjet]s
# and they share the same properties. using a class enforces that a bit
# plus autocompletion is better
class TestCase {
    [string]$Name
    [object]$In
}

# this cell is all example cases
$Samples = @(
    [TestCase]@{
        Name = 'TrueArray.Empty'
        In = @()
    }
    [TestCase]@{
        Name = 'String.TrueBlank'
        In = ''
    }
    [TestCase]@{
        Name = 'String.Space'
        In = '  '
    }
    [TestCase]@{
        Name = 'String.WithNonWhitespaceValue'
        In = ' 4'
    }
    [TestCase]@{
        Name = 'TrueNull'
        In = $Null
    }
    [TestCase]@{
        Name = 'Whitespace'
        In = "`n`t   `n"
    }
    [TestCase]@{
        Name = 'Variable.DoesNotExist'
        In = $ValueDoesNotExist
    }
    [TestCase]@{
        Name = 'Member.DoesNotExist'
        In = $Profile.FakeProp
    }
)


In [10]:
$samples | ForEach-Object {
    Test-IsBlank $_.In
        | Add-Member -NotePropertyMembers @{ Name = $_.Name } -pass -force -ea 'ignore'
} | Ft -auto


IsTrueNull IsEmpty IsTrueEmptyStr IsBlank     Length RawValue AsBool Name
---------- ------- -------------- -------     ------ -------- ------ ----
     False    True          False    True         15 {}         True TrueArray.Empty
     False    True           True    True <EmptyStr>            True String.TrueBlank
     False   False          False    True          2            True String.Space
     False   False          False   False          2  4        False String.WithNonWhitespaceValue
      True    True          False    True     <null>            True TrueNull
     False   False          False    True          6 …          True Whitespace
      True    True          False    True     <null>            True Variable.DoesNotExist
      True    True          False    True     <null>            True Member.DoesNotExist

